In [87]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [88]:
from fastai.vision import *
from fastai.datasets import *

In [89]:
PATH = Path('/home/jupyter/.fastai/data/pascal')

In [90]:
trn_j = json.load((PATH/'pascal_train2007.json').open())

In [91]:
IMAGES, ANNOTATIONS, CAT = ['images','annotations', 'categories']

In [92]:
FILE_NAME, ID, IMG_ID, CAT_ID, BBOX = 'file_name', 'id', 'image_id', 'category_id', 'bbox'

In [93]:
cats = {o[ID]:o['name'] for o in trn_j[CAT]}

In [94]:
trn_fns = {o[ID]: o[FILE_NAME] for o in trn_j[IMAGES]}

In [95]:
trn_ids = [o[ID] for o in trn_j[IMAGES]]

In [100]:
JPEGS = 'VOCdevkit/VOC2007/JPEGImages'

In [101]:
IMG_PATH = PATH/JPEGS

Above is getting things in order

In [102]:
def hw_bb(bb):
    return np.array([bb[1], bb[0], bb[1]+bb[3], bb[0]+bb[2]])

In [103]:
trn_anno = collections.defaultdict(lambda: []) #note this one

In [104]:
for o in trn_j[ANNOTATIONS]:
    if not o['ignore']:
        bb = o[BBOX]
        bb = hw_bb(bb)
        trn_anno[o[IMG_ID]].append((bb, o[CAT_ID]))

In [105]:
def get_lrg(b):
    if not b: raise Exception()
    b = sorted(b, key = lambda x: np.product(x[0][-2:] - x[0][:2]), reverse = True)
    return b[0]
    

In [106]:
trn_lrg_anno = collections.defaultdict(lambda :[])

In [107]:
trn_lrg_anno = {a: get_lrg(b) for a,b in trn_anno.items()}

In [108]:
CSV = PATH/'tmp/lrg.csv'

In [109]:
bs = 64

In [110]:
df = pd.read_csv(CSV); df.head()

,fn,cat
0,000012.jpg,car
1,000017.jpg,horse
2,000023.jpg,person
3,000026.jpg,car
4,000032.jpg,aeroplane


In [113]:
tfms = get_transforms()

In [115]:
data = (ImageList.from_csv(PATH, 'tmp/lrg.csv', folder = JPEGS)
       .split_by_rand_pct(0.2)
       .label_from_df()
       .transform(tfms, resize_method = ResizeMethod.SQUISH, size = 224)
       .databunch()
       .normalize(imagenet_stats))

In [119]:
arch = models.resnet34

In [120]:
def accuracy_long(input, targs):
    n = targs.shape[0]
    input = input.argmax(dim=-1).view(n, -1)
    targs = targs.view(n,-1).long()
    return (input==targs).float().mean()

In [121]:
learner = cnn_learner(data, arch, metrics = accuracy_long)